In [1]:
!pip install -q transformers datasets scikit-learn torch
!pip uninstall -y wandb

import os
os.environ["WANDB_DISABLED"] = "true"
import pandas as pd
import numpy as np
import re
import torch
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback,
    pipeline
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Load and map labels
from google.colab import files
uploaded = files.upload()
data = pd.read_csv("intent_data_ad_log_pro.csv")
label_map = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_map)
data.to_csv("my_data_mapped.csv", index=False)

dataset = load_dataset('csv', data_files="my_data_mapped.csv", split='train')

tokenizer_cls = AutoTokenizer.from_pretrained("bert-base-uncased")
model_cls = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

def tokenize_func(example):
    return tokenizer_cls(example["input"], padding="max_length", truncation=True)

tokenized = dataset.map(tokenize_func)
tokenized = tokenized.train_test_split(test_size=0.2)
train_valid = tokenized["train"].train_test_split(test_size=0.1, seed=42)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, preds), "f1": f1_score(labels, preds, average="weighted")}

args = TrainingArguments(
    output_dir="./intent-model",
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model_cls,
    args=args,
    train_dataset=train_valid["train"],
    eval_dataset=train_valid["test"],
    tokenizer=tokenizer_cls,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()


Saving intent_data_ad_log_pro.csv to intent_data_ad_log_pro.csv


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2070 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-3-7202356c8c47>:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.001269,1.000000,1.000000
2,No log,0.000466,1.000000,1.000000
3,0.034500,0.000293,1.000000,1.000000
4,0.034500,0.000234,1.000000,1.000000
5,0.034500,0.000217,1.000000,1.000000


TrainOutput(global_step=935, training_loss=0.018617750090711257, metrics={'train_runtime': 759.1202, 'train_samples_per_second': 9.814, 'train_steps_per_second': 1.232, 'total_flos': 1960194962073600.0, 'train_loss': 0.018617750090711257, 'epoch': 5.0})

In [5]:
intent_detector = pipeline("text-classification", model="./intent-model/checkpoint-935", tokenizer=tokenizer_cls)


Device set to use cuda:0


In [6]:
def parse_user_input(text):
    text = text.lower()
    age_match = re.search(r'(?:i am|i\'m)?\s*(\d+)\s*(?:year[-\s]?old|years?\s?old)', text)
    age = int(age_match.group(1)) if age_match else 30

    weight_match = re.search(r'(\d+\.?\d*)\s*kg', text)
    weight = float(weight_match.group(1)) if weight_match else 60.0

    height_match = re.search(r'(\d+\.?\d*)\s*cm', text)
    height = float(height_match.group(1)) if height_match else 160.0

    sex = 'female' if 'woman' in text or 'female' in text else 'male'

    activity_levels = ['sedentary', 'light', 'moderate', 'active', 'very active']
    activity_level = next((level for level in activity_levels if level in text), 'moderate')

    if 'lose' in text:
        goal = 'lose'
    elif 'gain' in text or 'bulk' in text:
        goal = 'gain'
    else:
        goal = 'maintain'

    match_weight_change = re.search(r'(lose|gain)\s*(\d+\.?\d*)\s*kg', text)
    target_weight_change_kg = float(match_weight_change.group(2)) if match_weight_change else 0

    match_duration = re.search(r'in\s*(\d+)\s*(week|month|day)', text)
    if match_duration:
        value, unit = int(match_duration.group(1)), match_duration.group(2)
        if 'day' in unit:
            duration_weeks = value / 7
        elif 'month' in unit:
            duration_weeks = value * 4
        else:
            duration_weeks = value
    else:
        duration_weeks = 0

    return {
        "weight_kg": weight,
        "height_cm": height,
        "age": age,
        "sex": sex,
        "activity_level": activity_level,
        "goal": goal,
        "target_weight_change_kg": target_weight_change_kg,
        "duration_weeks": duration_weeks
    }

def calculate_caloric_needs(weight_kg, height_cm, age, sex, activity_level, goal,
                             target_weight_change_kg=0, duration_weeks=0):
    if sex.lower() == 'male':
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age + 5
    elif sex.lower() == 'female':
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age - 161
    else:
        raise ValueError("Invalid sex")

    activity_multipliers = {
        'sedentary': 1.2, 'light': 1.375, 'moderate': 1.55, 'active': 1.725, 'very active': 1.9
    }

    tdee = bmr * activity_multipliers.get(activity_level.lower(), 1.55)
    daily_adjustment = 0
    if goal in ['lose', 'gain'] and target_weight_change_kg > 0 and duration_weeks > 0:
        kcal_change = 7700 * target_weight_change_kg
        daily_adjustment = kcal_change / (duration_weeks * 7)
        if goal == 'lose':
            daily_adjustment = -daily_adjustment

    return {
        'BMR': round(bmr, 2),
        'TDEE': round(tdee, 2),
        'Daily Caloric Adjustment': round(daily_adjustment, 2),
        'Recommended Daily Calories': round(tdee + daily_adjustment, 2)
    }


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "Soorya03/Llama-3.2-1B-Instruct-FitnessAssistant"
tokenizer_gen = AutoTokenizer.from_pretrained(model_id)
model_gen = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
fitness_pipe = pipeline("text-generation", model=model_gen, tokenizer=tokenizer_gen)


tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/926 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
def route_input(user_text):
    # Detect intent
    result = intent_detector(user_text)[0]
    label = int(result['label'].split("_")[-1])

    if label == 0:
        return f"[LOG] User log stored: '{user_text}'"

    elif label == 1:
        parsed = parse_user_input(user_text)
        calcs = calculate_caloric_needs(**parsed)
        return "\n".join([f"{k}: {v}" for k, v in calcs.items()])

    elif label == 2:
        reply = fitness_pipe(user_text, max_new_tokens=200, do_sample=True, temperature=0.7)
        return reply[0]['generated_text']

    else:
        return "Sorry, I couldn't understand the request."


In [21]:
user_input = "I'm 58, male, 96 kg, 178 cm, moderately active"
print(route_input(user_input))


BMR: 1927.5
TDEE: 2987.62
Daily Caloric Adjustment: 0
Recommended Daily Calories: 2987.62


In [19]:
user_input = "What exercises are good for building glutes"
print(route_input(user_input))

What exercises are good for building glutes and a strong bottom?

Exercises like squats, lunges, glute bridges, and hip thrusts are excellent for targeting the glutes. Additionally, exercises that work the glutes from other angles, such as step-ups and bodyweight squats, can also be effective.

In addition to these exercises, maintaining a healthy weight and engaging in regular cardio will help to build and tone the glutes. It's also essential to regularly stretch and foam roll the glutes to help reduce muscle soreness and promote recovery.

To get the most out of these exercises, focus on proper form and technique, and listen to your body and stop if you experience any pain or discomfort. With consistent effort and dedication, you can build strong, toned glutes and a confident bottom.

**Glute-Focused Exercises:**

1. Squats
2. Lunges
3. Glute bridges
4. Hip thrusts
5. Step-ups
6. Bodyweight squats
7. Dead


In [4]:
user_input = "Had chicken and chips for dinner."
print(route_input(user_input))

NameError: name 'route_input' is not defined

In [11]:
import shutil
shutil.make_archive('intent-model', 'zip', './intent-model/checkpoint-935')


'/content/intent-model.zip'

In [13]:
files.download('intent-model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
# ✅ 7. Gradio App
!pip install gradio
import gradio as gr
with gr.Blocks() as demo:
    gr.Markdown("## 🏋️‍♀️ Your Fitness Assistant")
    user_id = gr.Textbox(label="User ID", value="user1")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Message")
    send = gr.Button("Send")

    def respond(user_id, msg, chat_history):
        response = route_input(user_id, msg)
        chat_history.append((msg, response))
        return "", chat_history

    send.click(respond, [user_id, msg, chatbot], [msg, chatbot])

demo.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00


<ipython-input-3-b536bc757872>:7: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://077161b2bc6d57051c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
